# Τεχνητή Νοημοσύνη: Εργαστηριακή Άσκηση 2
---



Ο στόχος της εργασίας είναι η κατασκευή ενός συστήματος προτάσεων (Recommendation System) για ταινίες. Οι προτάσεις αυτές θα πηγάζουν τόσο από τα χαρακτηριστικά της ταινίας όσο και από ορισμένες αξιολογήσεις του κάθε χρήστη.

Στα δεδομένα της άσκησης περιλαμβάνονται ένα αρχείο με το όνομα movies_metadata.csv το όποιο περιέχει τα χαρακτηριστικά κάθε ταινίας όπως θέμα, σκηνοθέτης, ηθοποιοί, λέξεις κλειδιά κ.α. από το imdb, τα αρχεία user1.csv και user2.csv τα όποια περιέχουν αξιολογήσεις για 2 διαφορετικούς χρήστες καθώς και το αρχείο relative_movies.csv, που περιέχει 10 όμοιες ταινίες για ορισμένους τυχαία επιλεγμένους τίτλους.


---
#Κώδικας για κατασκευή περιβάλλοντος εργασίας

Κώδικας για να γίνει Mount to Google Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')
import os
os.listdir('/gdrive/My Drive')
path='/gdrive/My Drive/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


Κώδικάς για να εγκατασταθεί το Swi-Prolog και το pyswip.<br>

***Αυτά χρειάζονται σε κάθε περιβάλλον λειτουργίας, αν δεν έιναι ήδη  εγκατεστημένα.***

In [2]:
#install swi-prolog
!sudo apt-get install software-properties-common
!sudo apt-add-repository ppa:swi-prolog/stable
!sudo apt-get update
!sudo apt-get install swi-prolog

Reading package lists... Done
Building dependency tree       
Reading state information... Done
software-properties-common is already the newest version (0.96.24.32.11).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
 Comprehensive Prolog implementation with extensive libraries and development tools.   Primarily targetted at teaching, RDF processing and web-related tasks, such as creating web services or analysing web content.
 More info: https://launchpad.net/~swi-prolog/+archive/ubuntu/stable
Press [ENTER] to continue or Ctrl-c to cancel adding it.

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com

In [3]:
#install pyswip
!pip install pyswip

---

# **Μέρος 1: Μελέτη των Metadata, Δημιουργία κόσμου και των βασικών queries.**


In [7]:
# Η βιβλιοθήκη pandas είναι χρήσιμη για την εργασία με τέτοια δεδομένα
import pandas as pd 
# Διάβασμα του αρχείου 'movie_metadata.csv' 
data = pd.read_csv("movie_metadata.csv") 
#Στο csv υπαρχούν κελία με nan τιμές
#Στις θέσεις αυτές βάζουμε 'UNK' πράγμα που  κάνουμε με την παρακάτω συνάρτηση
data.fillna("UNK", inplace=True)
# Preview the first 5 lines of the loaded data 
data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723,178,0,855,Joel David Moore,1000,7.60506e+08,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054,English,USA,PG-13,2.37e+08,2009,936,7.9,1.78,33000
1,Color,Gore Verbinski,302,169,563,1000,Orlando Bloom,40000,3.09404e+08,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238,English,USA,PG-13,3e+08,2007,5000,7.1,2.35,0
2,Color,Sam Mendes,602,148,0,161,Rory Kinnear,11000,2.00074e+08,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994,English,UK,PG-13,2.45e+08,2015,393,6.8,2.35,85000
3,Color,Christopher Nolan,813,164,22000,23000,Christian Bale,27000,4.48131e+08,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701,English,USA,PG-13,2.5e+08,2012,23000,8.5,2.35,164000
4,UNK,Doug Walker,UNK,UNK,131,UNK,Rob Walker,131,UNK,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,UNK,0,UNK,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,UNK,UNK,UNK,UNK,UNK,UNK,12,7.1,UNK,0


In [0]:
from pyswip import Prolog

In [0]:
def clean_text(text):
    text = text.replace(u'\xa0', u'')
    text = text.replace(u"'", u'')
    text = text.replace(u'"', u'')
    return text

##Ερώτημα 1α
Κατασκευάζουμε τον κόσμο του προβλήματος.

In [0]:
#create World
#Ορίζουμε τον κόσμο μας
prolog = Prolog()
#και σβήνουμε ότι υπήρχε αποθηκευμένο από πρίν
prolog.query('make')

#Για κάθε row του πίνακα φτιάχνουμε τα κατηγοτήματα που θέλουμε να αποθηκέυσουμε
#αρχικά σε μια λίστα με το όνομα literals
literals = []
movie_score = {}
for row in data.itertuples(index=True, name='Pandas'):
    movie_title = clean_text(getattr(row, 'movie_title'))
    director = clean_text(getattr(row, 'director_name'))
    actor1 = clean_text(getattr(row, 'actor_1_name'))
    actor2 = clean_text(getattr(row, 'actor_2_name'))
    actor3 = clean_text(getattr(row, 'actor_3_name'))
    language = clean_text(getattr(row, 'language'))
    color = clean_text(getattr(row, 'color'))
    country = clean_text(getattr(row, 'country'))
    imdb_score = str(getattr(row, 'imdb_score'))
    title_year = str(getattr(row, 'title_year'))

    
    movie_score[movie_title] = 0
    
    if movie_title != "UNK":
        if director != "UNK":
            literals.append("director('"+ movie_title +"','" + director + "')")
        if actor1 != "UNK":
            literals.append("actors('"+ movie_title +"','" + actor1 + "')")
        if actor2 != "UNK":
            literals.append("actors('"+ movie_title +"','" + actor2 + "')")
        if actor3 != "UNK":
            literals.append("actors('"+ movie_title +"','" + actor3 + "')")
        if language != "UNK":
            literals.append("language('"+ movie_title +"','" + language + "')")
        if country != "UNK":
            literals.append("country('"+ movie_title +"','" + country + "')")
        if imdb_score != "UNK":
            literals.append("imdb_score('"+ movie_title +"','" + imdb_score + "')")
        if title_year != "UNK":
            literals.append("title_year('"+ movie_title +"','" + title_year + "')")
        for genre in getattr(row, 'genres').split("|"):
            literals.append("genre('"+ movie_title +"','"+ genre +"')")
        for plot in getattr(row, 'plot_keywords').split("|"):
            if plot != "UNK":
                literals.append("plot('"+ movie_title +"','"+ clean_text(plot) +"')")

#Η Prolog θέλει τα κατηγορήματά της με την σειρά 
literals.sort()
for literal in literals:
    prolog.assertz(literal)

## Ερώτημα 1β

Kάνουμε consult ένα έτοιμό αρχείο, όπου έχουμε δημιουργήσει απλούς κανόνες οι οποίοι θα βρίσκουν όλες τις ταινίες με βάση τα ζητούμενα χαρακτηριστικά.

Σημειώνεται ότι έγινε αλλαγή στην κλιμακωσιμότητα για τα queries που αφορούν την πλοκή και το θέμα (από 1 εώς 5), ενώ για τους ηθοποιούς παρέμεινε η ίδια (από 1 εώς 3) καθώς δεν είναι δυνατή η αλλαγή της, μιας και για κάθε ταινία στο αρχείο csv περιέχονται μόνο 3 ηθοποιοί. Ακόμα, προστέθηκαν περισσότερα ερωτήματα, τα οποία αφορούν το έτος κυκλοφορίας και την βαθμολογία στο imdb.

In [0]:
prolog.consult('db.pl')

# **2ο Μέρος: Recommendation System με βάση μόνο τα χαρακτηριστικά των ταινιών.**

Στο σημείο αυτό με βάση τους κανόνες που κατασκευάστηκαν στο Μέρος 1, κατασκευάζονται κλιμακούμενα ερωτήματα για την ομοιότητα ταινιών.

In [0]:
prolog.assertz('(find_similar_movies_5(X, Y) :- find_similar_director(X, Y), find_similar_language(X, Y), find_similar_genre_2(X, Y) )')
prolog.assertz('(find_similar_movies_4(X, Y) :- find_similar_actors_3(X, Y), find_similar_genre_1(X, Y), not(find_similar_movies_5(X, Y)) )')
prolog.assertz('(find_similar_movies_3(X, Y) :- find_similar_genre_3(X, Y), find_similar_language(X, Y), not(find_similar_movies_4(X, Y)) )')
prolog.assertz('(find_similar_movies_2(X, Y) :- find_similar_language(X, Y), find_similar_plot_2(X, Y), not(find_similar_movies_3(X, Y)) )')
prolog.assertz('(find_similar_movies_1(X, Y) :- find_similar_genre_2(X, Y), not(find_similar_movies_2(X, Y)) )')

In [0]:
def simple_recommender(movie):

    q = prolog.query("find_similar_movies_5('" + movie +"',M)")
    ans5 = []
    for soln in q:
        ans5.append(soln["M"])
    ans = set(ans5)
    
    q = prolog.query("find_similar_movies_4('" + movie +"',M)")
    ans4 = []
    for soln in q:
        if soln["M"] not in ans:
            ans4.append(soln["M"])
            ans.add(soln["M"])

    q = prolog.query("find_similar_movies_3('" + movie +"',M)")
    ans3 = []
    for soln in q:
        if soln["M"] not in ans:
            ans3.append(soln["M"])
            ans.add(soln["M"])

    q = prolog.query("find_similar_movies_2('" + movie +"',M)")
    ans2 = []
    for soln in q:
        if soln["M"] not in ans:
            ans2.append(soln["M"])
            ans.add(soln["M"])

    q = prolog.query("find_similar_movies_1('" + movie +"',M)")
    ans1 = []
    for soln in q:
        if soln["M"] not in ans:
            ans1.append(soln["M"])
            ans.add(soln["M"])

    #κάνουμε κάποιο Processing στα παραπάνω αποτελέσματα και επιστρέφουμε τις "όμοιες" ταινίες με φθίνουσα σειρά
    return list(set(ans5))[:10], list(set(ans4))[:10], list(set(ans3))[:10], list(set(ans2))[:10], list(set(ans1))[:10]

##Αποτελέσματα

Στο σημείο αυτό θα ζητήσουμε από το σύστημα να μας επιστρέψει ορισμένα αποτελέσματα για ταινίες για να εξετάσουμε την απόδοσή του.


In [14]:
simple_recommender('Pirates of the Caribbean: Dead Mans Chest')

(['The Lone Ranger'],
 ['Pirates of the Caribbean: At Worlds End',
  'Pirates of the Caribbean: The Curse of the Black Pearl'],
 ['Atlantis: The Lost Empire',
  'The Golden Child',
  'Mighty Joe Young',
  'X-Men: The Last Stand',
  'The Mummy Returns',
  'Underworld: Rise of the Lycans',
  'Beowulf',
  'Krull',
  'Journey to the Center of the Earth',
  'Destiny'],
 [],
 ['Toy Story 2',
  'King Kong',
  'Howard the Duck',
  'Transformers: Revenge of the Fallen',
  'The Bounty',
  'The Prince of Egypt',
  'The Promise',
  'Frozen',
  'Battleship',
  'Harry Potter and the Prisoner of Azkaban'])

In [15]:
simple_recommender("Indiana Jones and the Temple of Doom")

(['The Adventures of Tintin',
  'Indiana Jones and the Kingdom of the Crystal Skull',
  'The Lost World: Jurassic Park',
  'Indiana Jones and the Last Crusade',
  'Raiders of the Lost Ark'],
 [],
 [],
 [],
 ['Atlantis: The Lost Empire',
  'King Kong',
  'The Golden Child',
  'Howard the Duck',
  'Transformers: Revenge of the Fallen',
  'The Bounty',
  'The Mummy Returns',
  'Underworld: Rise of the Lycans',
  'Krull',
  'Battleship'])

In [16]:
simple_recommender("Species")

(['Dantes Peak', 'The November Man'],
 [],
 ['Dawn of the Dead',
  'Skyline',
  'Anacondas: The Hunt for the Blood Orchid',
  'The Covenant',
  '10 Cloverfield Lane',
  'Doomsday',
  'Deep Impact',
  'X-Men: The Last Stand',
  'Grindhouse',
  'Resident Evil'],
 ['The Day the Earth Stood Still',
  'Titan A.E.',
  'The Andromeda Strain',
  'Splice'],
 ['Atlantis: The Lost Empire',
  'Def-Con 4',
  'Jaws 2',
  'Blue Streak',
  'Pride and Prejudice and Zombies',
  'Howard the Duck',
  'The Boy',
  'Transformers: Revenge of the Fallen',
  'Subconscious',
  'Fast Five'])

##Αξιολόγηση

Για την αξιολόγηση των αποτελεσμάτων των συστάσεων, χρησιμοποιήθηκε το αρχείο relative_movies.csv (με ορισμένες ταινίες και τα 10 πρώτα αποτελέσματά τους σε φθίνουσα σειρά ομοιότητας). Για τη σύγκρισή αυτή χρησιμοποιείται η συνάρτηση ομοιότητας συνόλων Jaccard

$$jaccard(A,B)=\frac{|A\cap B|}{|A\cup B|}$$ 

Αξίζει να σημειωθεί ότι η παραπάνω σύγκριση δεν είναι απόλυτη για τον λόγο ότι οι συναρτήσεις ομοιότητας μπορεί να είναι διαφορετικές (διαφορετικά ορίζει ο καθένας της ομοιότητα μεταξύ ταινιών) καθώς επίσης αν μια ταινία δεν έχει πολύ σχετικές προτάσεις τότε οι πιο γενικές συναρτήσεις ομοιότητας θα έχουν πολλά αποτελέσματα τα όποια η Prolog θα επιστρέψει με την σειρά που τα βρίσκει στην βάση, όποτε τότε δεν έχει ιδιαίτερο νόημα η σύγκριση σε αυτήν την περίπτωση.
Η ομοιότητα των ταινιών είναι βοηθητική και σκοπό έχει να σας βοηθήσει να κατανοήσετε την κατεύθυνση της ασκήσης και όχι για να καταφέρετε την πλήρη ταύτιση των συστάσεων.

In [0]:
def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1) + len(set2) - intersection
    return intersection / union

In [0]:
#διάβασμα του αρχείου relative_movies.csv
data_relative = pd.read_csv("relative_movies.csv")

relative = {}
for row in data_relative.itertuples(index=True, name='Pandas'):
    movie_title = clean_text(getattr(row, 'movie_title'))

    rel = []
    if movie_title != "UNK":
      for m in getattr(row, 'relative_movies').split(","):
            if m != "UNK":
                rel.append( m )

    relative[movie_title] = rel

In [0]:
# Δημιουργία λιστών για την αξιολόγηση της ταινίας movie
def simple_compare(movie):
  l = simple_recommender(movie)
  flat_list = [item for sublist in l for item in sublist][:10]
  print(jaccard_similarity(flat_list, relative[movie]))

In [19]:
simple_compare("Species")

0.1111111111111111


# **3ο Μέρος: Recommendation System Με βάση τις προτιμήσεις-αξιολογήσεις του χρήστη**


##User 1

Εργαζόμαστε με τα αρχεία user1.csv και user2.csv, τα όποια περιέχουν αξιολογήσεις (από 1 μέχρι 5) για ταινίες από 2 διαφορετικούς χρήστες. Οι συστάσεις τώρα θα λαμβάνουν υπόψιν και τις προτιμήσεις του χρήστη, οι όποιες θα εξάγονται από τις αξιολογήσεις που έχει κάνει μέχρι στιγμής. 

Αρχικά μελετάμε τις αξιολογήσεις κάθε χρήστη για να καταλάβουμε την δομή και τις πληροφορίες κάθε αρχείου.

In [20]:
user = "user1.csv"
ratings = pd.read_csv(user) 

# Preview the first 5 lines of the loaded data 
ratings.head()

,movie_title,rating
0,Heavy Metal,4
1,The Croods,5
2,Watchmen,4
3,The Matrix,4
4,The Emperors New Groove,5


**Κατασκευάζουμε τα scores για κάθε ταινία με βάση τις προηγούμενες αξιολογήσεις του χρήστη**

Αν $rated\_movies$ είναι το σύνολο ταινιών που έχει αξιολογήσει ο χρήστης, τότε τα recommendations που θα λαμβάνει ο χρήστης θα είναι οι ταινίες που είναι τουλάχιστον similar_1 με τουλάχιστον μία ταινία που έχει βαθμολογήσει ο χρήστης, σε φθίνουσα σειρά recommendation_score, όπου:

$$recommendation\_score(m_1)=\sum_{m_2\in rated\_movies} similarity\_weight(m_1,m_2)\cdot voting\_weight(m_2)$$

In [0]:
similarity_weights = {"similarity_5": 15, "similarity_4":7, "similarity_3":3, "similarity_2":2, "similarity_1":1}
rating_weight = {5: 10, 4: 5, 3: 0, 2: -5, 1: -10}

movie_score = {m: 0 for m in  movie_score}
for row in ratings.itertuples(index=True, name='Pandas'):
    movie = clean_text(getattr(row, 'movie_title'))
    rating = getattr(row, 'rating')    
    
    q = prolog.query("find_similar_movies_5('" + movie + "',M)")
    for soln in q:
        movie_score [soln["M"]] += similarity_weights['similarity_5'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()
    
    q = prolog.query("find_similar_movies_4('" + movie + "',M)")
    for soln in q:
        movie_score [soln["M"]] += similarity_weights['similarity_4'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()

    q = prolog.query("find_similar_movies_3('" + movie + "',M)")
    c = 0
    for soln in q:
        c += 1
        if c>10: break
        movie_score [soln["M"]] += similarity_weights['similarity_3'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()

    q = prolog.query("find_similar_movies_2('" + movie + "',M)")
    c = 0
    for soln in q:
        c += 1
        if c>5: break
        movie_score [soln["M"]] += similarity_weights['similarity_2'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()

    q = prolog.query("find_similar_movies_1('" + movie + "',M)")
    c = 0
    for soln in q:
        c += 1
        if c>5: break
        movie_score [soln["M"]] += similarity_weights['similarity_1'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()


**Στο σημείο αυτό κατασκευάζουμε την νέα συνάρτηση η όποια θα επιστρέφει τις προτεινόμενες ταινίες συνυπολογίζοντας τις αξιολογήσεις του χρήστη.**

In [0]:
def recommender(movie_score):
    movies_score = {k: v for k, v in sorted(movie_score.items(), key=lambda item: item[1], reverse = True)}
    return [m for m in movies_score][:10]

In [23]:
recommender(movie_score)

['Teenage Mutant Ninja Turtles',
 'Trance',
 '20,000 Leagues Under the Sea',
 'Star Wars: Episode III - Revenge of the Sith',
 'Star Wars: Episode I - The Phantom Menace',
 'Chicken Little',
 'Space Chimps',
 'Cats Dont Dance',
 'Star Wars: Episode IV - A New Hope',
 'Pirates of the Caribbean: At Worlds End']

Μελετώντας το αρχείο user1.csv, παρατηρούμε πως ο χρήστης 1 έχει αξιολογήσει με 5 κυρίως ταινίες που ανήκουν στις κατηγορίες family, adventure, sci-fi/fantasy. Βλέπουμε πράγματι πως οι προτάσεις του συστήματος συστάσεων είναι ταινίες που ανήκουν σε αυτές τις κατηγορίες. Φυσικά υπάρχουν και πολλά ακόμη κοινά χαρακτηριστικά μεταξύ των ταινιών με υψηλές αξιολογήσεις και αυτών που προτείνει το σύστημά μας. 

Σε κάθε περίπτωση, το σύστημά μας διαφέρει από αυτό τους 2ου μέρους, μιας και τότε παράγονταν συστάσεις από μία ταινία και με βάση τα χαρακτηριστικά της, ενώ τώρα παράγονται συστάσεις για έναν χρήστη, συνδυάζοντας ταινίες - αξιολογήσεις, και λαμβάνονται υπόψη τόσο τα χαρακτηριστικά αυτών των ταινιών, όσο και η βαθμολογία του χρήστη για κάθε μία. 

Με λίγα λόγια, το σύστημα συστάσεων τώρα είναι αναβαθμισμένο και αφορά κάθε φορά έναν συγκεκριμένο χρήστη και τις προτιμήσεις του και όχι μια συγκεκριμένη ταινία.

##User 2

In [0]:
user = "user2.csv"
ratings2 = pd.read_csv(user) 

In [0]:
similarity_weights = {"similarity_5": 15, "similarity_4":7, "similarity_3":3, "similarity_2":2, "similarity_1":1}
rating_weight = {5: 10, 4: 5, 3: 0, 2: -5, 1: -10}

movie_score = {m: 0 for m in  movie_score}
for row in ratings2.itertuples(index=True, name='Pandas'):
    movie = clean_text(getattr(row, 'movie_title'))
    rating = getattr(row, 'rating')    
    
    q = prolog.query("find_similar_movies_5('" + movie + "',M)")
    for soln in q:
        movie_score [soln["M"]] += similarity_weights['similarity_5'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()
    
    q = prolog.query("find_similar_movies_4('" + movie + "',M)")
    for soln in q:
        movie_score [soln["M"]] += similarity_weights['similarity_4'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()

    q = prolog.query("find_similar_movies_3('" + movie + "',M)")
    c = 0
    for soln in q:
        c += 1
        if c>10: break
        movie_score [soln["M"]] += similarity_weights['similarity_3'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()

    q = prolog.query("find_similar_movies_2('" + movie + "',M)")
    c = 0
    for soln in q:
        c += 1
        if c>5: break
        movie_score [soln["M"]] += similarity_weights['similarity_2'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()

    q = prolog.query("find_similar_movies_1('" + movie + "',M)")
    c = 0
    for soln in q:
        c += 1
        if c>5: break
        movie_score [soln["M"]] += similarity_weights['similarity_1'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()

In [26]:
recommender(movie_score)

['Kiss of Death',
 'My Soul to Take',
 '2 Guns',
 '88 Minutes',
 'Righteous Kill',
 'Kiss the Girls',
 '12 Rounds',
 '15 Minutes',
 '16 Blocks',
 'Blood Work']

Μελετώντας το αρχείο user2.csv, παρατηρούμε πως ο χρήστης 2 έχει αξιολογήσει με 5 κυρίως ταινίες που ανήκουν στις κατηγορίες thriller, mystery, drama. Και πάλι, βλέπουμε πως οι προτάσεις του συστήματος συστάσεων είναι ταινίες που ανήκουν σε αυτές τις κατηγορίες. 

##My user

---
<center><h2>Κατασκευή των Δικών σας Users</h2></center>

---

In [0]:
import csv
import numpy as np

def create_my_user(liked, notLiked, filename, num_of_liked =20, num_of_notliked = 20):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["movie_title","rating"])
        liked = np.random.choice(liked, num_of_liked)
        notliked = np.random.choice(notLiked, num_of_notliked)
        for i in  liked:
            grade = np.random.choice([5,4,3], p = [0.5, 0.4, 0.1])
            writer.writerow([i, grade])
        for i in  notliked:
            grade = np.random.choice([1, 2, 3], p = [0.5, 0.4, 0.1])
            writer.writerow([i, grade])

In [0]:
#Κατασκευάστε τα κατηγορήματα για τις ταινίες που σας αρέσουν και για αυτές που δεν σας αρέσουν
#Αν μια ταινία ανήκει σε μια ευρίτερη κατηγορία κατεσκευάστε την πρώτα
prolog.assertz("(romance(X) :- genre(X, 'Romance'); genre(X, 'Comedy'); plot(X, 'love'))")
prolog.assertz("(anime(X) :- genre(X, 'Family'); genre(X, 'Animation'))")
prolog.assertz("(thriller(X) :- genre(X, 'Horror'); genre(X, 'Thriller'); genre(X, 'Mystery'); plot(X, 'alien'))")
prolog.assertz("(action(X) :- genre(X, 'Action'); genre(X, 'Crime');  plot(X, 'kill'))")
prolog.assertz("(notLiked(X) :- thriller(X); action(X))")
prolog.assertz("(liked(X) :- (romance(X); anime(X)), not(notLiked(X)))")

In [0]:
q = prolog.query("liked(M)")
ans = []
for soln in q:
    ans.append(soln["M"])
liked = list(set(ans))

q = prolog.query("notLiked(M)")
ans = []
for soln in q:
    ans.append(soln["M"])
notliked = list(set(ans))

create_my_user(liked, notliked, filename= "myuser.csv", num_of_liked = 20, num_of_notliked = 20)

In [0]:
user = "myuser.csv"
ratings3 = pd.read_csv(user) 

In [0]:
similarity_weights = {"similarity_5": 15, "similarity_4":7, "similarity_3":3, "similarity_2":2, "similarity_1":1}
rating_weight = {5: 10, 4: 5, 3: 0, 2: -5, 1: -10}

r = 0
movie_score = {m: 0 for m in  movie_score}
for row in ratings3.itertuples(index=True, name='Pandas'):
    movie = clean_text(getattr(row, 'movie_title'))
    rating = getattr(row, 'rating')    
    
    q = prolog.query("find_similar_movies_5('" + movie + "',M)")
    for soln in q:
        movie_score [soln["M"]] += similarity_weights['similarity_5'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()
    
    q = prolog.query("find_similar_movies_4('" + movie + "',M)")
    for soln in q:
        movie_score [soln["M"]] += similarity_weights['similarity_4'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()

    q = prolog.query("find_similar_movies_3('" + movie + "',M)")
    c = 0
    for soln in q:
        c += 1
        if c>10: break
        movie_score [soln["M"]] += similarity_weights['similarity_3'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()

    q = prolog.query("find_similar_movies_2('" + movie + "',M)")
    c = 0
    for soln in q:
        c += 1
        if c>5: break
        movie_score [soln["M"]] += similarity_weights['similarity_2'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()

    q = prolog.query("find_similar_movies_1('" + movie + "',M)")
    c = 0
    for soln in q:
        c += 1
        if c>5: break
        movie_score [soln["M"]] += similarity_weights['similarity_1'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    q.close()

In [32]:
recommender(movie_score)

['Juno',
 '13 Going on 30',
 'Bride Wars',
 'Girl, Interrupted',
 'See Spot Run',
 'Sliding Doors',
 'Bernie',
 '17 Again',
 'The Hours',
 'Young Adult']

Παρατηρούμε πως οι παραπάνω ταινίες είναι Κωμωδίες ή Κινούμενα Σχέδια ή Ρομαντικές Κομεντί, και άρα σύμφωνες με τις κατηγορίες που ορίσαμε ως liked. Επίσης, δε συναντάμε ταινίες Θρίλλερ ή Τρόμου, που ορίσαμε ως notLiked, ούτε ταινίες με "kill" στην πλοκή.